In [53]:
from IPython.display import display, HTML

display(HTML('''
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>
'''))

import folium
from folium.plugins import MarkerCluster
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import branca


# -----------------------------------------------------------------------------
# Read data from file into dataframe and perform basic cleaning

file_path = 'bunker_prices.xlsx'

# Read the Excel file into a DataFrame, setting the header row.
df = pd.read_excel(file_path, header=1)

# set name for first data column as "Location"
df.rename(columns={df.columns[1]: 'Location'}, inplace=True)

# remove first column which is empty
df = df.iloc[:, 1:]

#remove last 3 columns, which are broader date ranges 
df = df.iloc[:, :-3]

# remove any row that does not contain a location/fuel datapoint
# (Removing rows that do not contain '_')
df = df[df['Location'].str.contains('_', na=False)]

# -----------------------------------------------------------------------------
# perform more data cleaning

# Replace a cell value in the 'Location' column
df.loc[df['Location'] == 'Rotterdam_B30-VLSFO_Biofuels ', 'Location'] = 'Rotterdam_B30 VLSFO Biofuels'
df.loc[df['Location'] == 'Rotterdam_B30-LSG_Biofuels ', 'Location'] = 'Rotterdam_B30 LSG Biofuels'


# # Splitting the column into two
df[['Location', 'Fuel Type']] = df['Location'].str.split('_', expand=True)

# Move the last column to be the second column
cols = list(df.columns)
cols.insert(1, cols.pop(-1))
df = df[cols]

# Remove white spaces from the Fuel type column
df['Fuel Type'] = df['Fuel Type'].str.strip()



# -----------------------------------------------------------------------------
# split df into 3 based on fuel type

# Copy rows that include 'IFO' in the second column to a new DataFrame
df_ifo = df[df.iloc[:, 1]   == 'IFO']
df_vlsfo = df[df.iloc[:, 1] == 'VLSFO']
df_lsg = df[df.iloc[:, 1]   == 'LSG']

# Remove fuel type column from 3 fuel-specific dataframes
df_ifo.pop('Fuel Type')
df_vlsfo.pop('Fuel Type')
df_lsg.pop('Fuel Type')

# set all data as integer
# df_ifo = df_ifo.astype(int)
# df_vlsfo = df_vlsfo.astype(int)
# df_lsg = df_lsg.astype(int)


# Transpose 3 fuel dataframes
df_ifo = df_ifo.T
df_vlsfo = df_vlsfo.T
df_lsg = df_lsg.T

# -----------------------------------------------------------------------------
# use top row as df column names

#ifo
df_ifo.columns = df_ifo.iloc[0]
df_ifo = df_ifo[1:]

#vlsfo
df_vlsfo.columns = df_vlsfo.iloc[0]
df_vlsfo = df_vlsfo[1:]

#lsg
df_lsg.columns = df_lsg.iloc[0]
df_lsg = df_lsg[1:]




# Load the bunker location coordinates Excel file into a DataFrame
bunker_coordinates = pd.read_excel('Bunker_location_coordinates.xlsx')





In [54]:

# Ensure the 'Location' column is consistent across all dataframes
bunker_coordinates['Location'] = bunker_coordinates['Location'].str.strip()

# Function to merge fuel price data with coordinates
def merge_fuel_data_with_coords(fuel_df, bunker_coordinates):
    fuel_df = fuel_df.reset_index().rename(columns={'index': 'Date'})
    fuel_df = fuel_df.melt(id_vars=['Date'], var_name='Location', value_name='Price')
    merged_df = fuel_df.merge(bunker_coordinates, on='Location', how='left')
    return merged_df

# Merge data for each fuel type
df_ifo_merged = merge_fuel_data_with_coords(df_ifo, bunker_coordinates)
df_vlsfo_merged = merge_fuel_data_with_coords(df_vlsfo, bunker_coordinates)
df_lsg_merged = merge_fuel_data_with_coords(df_lsg, bunker_coordinates)

# Combine all fuel data
df_all_fuels = pd.concat([
    df_ifo_merged.assign(Fuel_Type="IFO"),
    df_vlsfo_merged.assign(Fuel_Type="VLSFO"),
    df_lsg_merged.assign(Fuel_Type="LSG")
])

# Drop rows with missing coordinates
df_all_fuels = df_all_fuels.dropna(subset=['Latitude', 'Longitude'])

# Convert Date column to datetime
df_all_fuels['Date'] = pd.to_datetime(df_all_fuels['Date'])

# Create interactive widgets for filtering
fuel_type_dropdown = widgets.Dropdown(
    options=["IFO", "VLSFO", "LSG"],  
    value="IFO",  
    description="Fuel Type:"
)

date_dropdown = widgets.Dropdown(
    options=sorted(df_all_fuels['Date'].dt.strftime('%Y-%m-%d').unique()),
    value=df_all_fuels['Date'].dt.strftime('%Y-%m-%d').min(),
    description="Date:"
)

# Output widget to display map
map_output = widgets.Output()

# Function to get color based on price (heatmap effect)
def get_price_color(price, min_price, max_price):
    if min_price == max_price:
        return "rgb(255, 255, 0)"  # Yellow if all prices are the same
    normalized = (price - min_price) / (max_price - min_price)
    r = int(255 * normalized)  
    g = int(255 * (1 - normalized))  
    return f"rgb({r},{g},0)"  

# Function to update map based on selected filters
def update_map(fuel_type, selected_date):
    with map_output:
        clear_output(wait=True)

        # Convert selected date to datetime
        selected_date = pd.to_datetime(selected_date)

        # Filter data based on selected fuel type and date
        filtered_df = df_all_fuels[
            (df_all_fuels['Date'] == selected_date) & (df_all_fuels["Fuel_Type"] == fuel_type)
        ]

        # Determine min and max price for the color scale
        if not filtered_df.empty:
            min_price = filtered_df['Price'].min()
            max_price = filtered_df['Price'].max()
            map_center = [filtered_df['Latitude'].mean(), filtered_df['Longitude'].mean()]
        else:
            min_price, max_price = 0, 1  
            map_center = [0, 0]  

        # Create the Folium map
        m = folium.Map(location=map_center, zoom_start=3)

        # Create a color scale (gradient bar) using branca
        colormap = branca.colormap.LinearColormap(
            colors=["green", "yellow", "red"],  
            vmin=min_price,
            vmax=max_price,
            caption="Fuel Price ($/ton)"
        )

        # Add the color scale to the map
        colormap.add_to(m)

        # Add price labels with small backgrounds
        for _, row in filtered_df.iterrows():
            background_color = get_price_color(row['Price'], min_price, max_price)

            price_label = folium.DivIcon(
                icon_size=(50, 15),
                icon_anchor=(25, 7),
                html=f"""
                <div style="background:{background_color}; color:black; padding:2px 5px; border-radius:10px;
                            font-size:10px; font-weight:bold; text-align:center;">
                    {row['Price']}
                </div>
                """
            )

            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                icon=price_label
            ).add_to(m)

        # Display the map in the Jupyter Notebook
        display(m)

# Set up interactive widgets
interactive_plot = widgets.interactive(update_map, fuel_type=fuel_type_dropdown, selected_date=date_dropdown)

# Display widgets and map output
display(fuel_type_dropdown, date_dropdown, map_output)

# Initial map render
update_map("IFO", df_all_fuels['Date'].dt.strftime('%Y-%m-%d').min())


Dropdown(description='Fuel Type:', options=('IFO', 'VLSFO', 'LSG'), value='IFO')

Dropdown(description='Date:', options=('2025-01-26', '2025-02-26', '2025-03-25', '2025-03-26', '2025-04-25', '…

Output()